GETTING THE API KEY

In [45]:
import os
from dotenv import load_dotenv

load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
MODEL = 'gemini-pro'
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

TEXT EXTRACTION FROM PDF

In [5]:
from PyPDF2 import PdfReader

In [7]:
dataset_path = 'G:/policy-booklet-0923.pdf'

def get_pdf_text(docs):
    text = ""
    pdf_reader = PdfReader(docs)
    for page in pdf_reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text.replace('\n', ' ')
    return text

extracted_text = get_pdf_text(dataset_path)

with open('G:/test/test.txt', 'a', encoding='utf-8') as file:
    file.write(extracted_text)
    

SCRAPING THE TEST.TEXT (PDF CONTENT) AND SPLITTING THE CONTENT

In [10]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [11]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)

In [13]:
path = 'G:/test'
loader = DirectoryLoader(path)
documents = loader.load_and_split(text_splitter)

In [14]:
documents

[Document(page_content='Your car insurance  policy booklet Page 2FAQs  3 Glossary  4 Making a claim  6 What your cover includes  8 Section 1: Liability  11 Section 2: Fire and theft  14 Section 3: Courtesy car  17 Section 4: Accidental damage  18 Section 5: Windscreen damage  20 Section 6: Personal benefits  21 Section 7: Motor Legal Cover  23 Section 8: Guaranteed Hire Car Plus  28 Section 9: Protected No Claim Discount  30Where you can drive  31 Losses we don’t cover  33 Other conditions you need to know about  36 How the policy works  37 Everything else  41 If you have a complaint  42 If you’re in an accident  43 How to get in touch  Back coverContentsWelcome to Churchill This booklet tells you about your car insurance About the policy The policy is made up of:  >This booklet. >Your car insurance details. >Your certificate (or certificates)   of motor insurance. If the policy includes Green Flag breakdown cover:  >Your breakdown cover and your car  insurance are part of the same pol

LOADING THE CONTENT IN A VECTOR STORE AND CREATING A KNOWLEDGE BASE

In [15]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch

In [16]:
vector_store = DocArrayInMemorySearch.from_documents(
    documents,
    embedding=GoogleGenerativeAIEmbeddings(model='models/embedding-001')
)

G:\Evaluating-RAG-Application\venv\lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [18]:
import pandas as pd

In [19]:
df = pd.DataFrame([d.page_content for d in documents], columns=['text'])
df.head()

,text
0,Your car insurance policy booklet Page 2FAQs ...
1,>The policy also includes the Green Flag poli...
2,You can find the exact meanings of these word...
3,if I leave my car unlocked or the keys in the...
4,>Breakdowns (if your policy includes Green Fl...


In [42]:
from giskard.rag import KnowledgeBase

knowledge_base = KnowledgeBase(df)

GENERATING THE SYNTHETIC TEST CASES

In [43]:
from giskard.rag import generate_testset

testset = generate_testset(
    knowledge_base,
    num_questions=60,
    agent_description='A chatbot answering questions about the provided context.'
)

2024-06-09 14:40:53,860 pid:3636 MainThread giskard.rag  INFO     Finding topics in the knowledge base.


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}